In [24]:
import pickle as pkl
import importlib
import jax.numpy as jnp
import numpy as np

# Import and reload the model
import model
importlib.reload(model)

physical_ei = model.generate_random_gt_params_ei(1)


print("Creating trainer...")
trainer = model.StraightAxon()
print("Trainer created successfully!")
print("Training with physical_ei:", physical_ei)
trainer.train(true_ei=physical_ei)

Number of newly added trainable parameters: 1. Total number of trainable parameters: 1
Number of newly added trainable parameters: 1. Total number of trainable parameters: 2
Number of newly added trainable parameters: 1. Total number of trainable parameters: 3
Number of newly added trainable parameters: 1. Total number of trainable parameters: 4
Added 1 external_states. See `.externals` for details.
Added 1500 recordings. See `.recordings` for details.
Added 1500 recordings. See `.recordings` for details.
Creating trainer...
Number of newly added trainable parameters: 1. Total number of trainable parameters: 1
Number of newly added trainable parameters: 1. Total number of trainable parameters: 2
Number of newly added trainable parameters: 1. Total number of trainable parameters: 3
Number of newly added trainable parameters: 1. Total number of trainable parameters: 4
Trainer created successfully!
Training with physical_ei: ({'axon_origin_dist': Array([18.83830807], dtype=float64), 'axon

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int64[]
It arose in the jnp.arange argument 'start'
The error occurred while tracing the function compute_ei_width at /Users/andrewlanpouthakoun/Desktop/Stats220/extracellular-jaxley/exp/loss_functions.py:10 for jit. This concrete value was not available in Python because it depends on the value of the argument max_time_offset.

See https://docs.jax.dev/en/latest/errors.html#jax.errors.ConcretizationTypeError